In [1]:
import pandas as pd
import numpy as np

male_df = pd.read_csv("C:/Users/JEONGHEE/Desktop/당뇨병플젝/male.csv")
female_df = pd.read_csv("C:/Users/JEONGHEE/Desktop/당뇨병플젝/female.csv")

# 남자

## 랜덤포레스트 

In [7]:
## 언더샘플링 안한 상태 

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report


drop_cols = ['DS1_ID', 'DS1_SEX', 'DS1_GLU0', 'DS1_HBA1C']

# 입력(X) / 타겟(y) 정의
X = male_df.drop(columns=drop_cols + ['target'])
y = male_df['target']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)

print("테스트 데이터 성능 평가:")
print(classification_report(y_test, y_pred, target_names=['비당뇨(0)', '당뇨(1)']))

테스트 데이터 성능 평가:
              precision    recall  f1-score   support

      비당뇨(0)       0.96      1.00      0.98      6991
       당뇨(1)       0.00      0.00      0.00       321

    accuracy                           0.96      7312
   macro avg       0.48      0.50      0.49      7312
weighted avg       0.91      0.96      0.93      7312



C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [11]:
## 언더샘플링만 적용

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.utils import resample

drop_cols = ['DS1_ID', 'DS1_SEX', 'DS1_GLU0', 'DS1_HBA1C']

# 입력(X) / 타겟(y) 정의
X = male_df.drop(columns=drop_cols + ['target'])
y = male_df['target']


df_all = X.copy()
df_all['target'] = y

df_majority = df_all[df_all['target'] == 0]
df_minority = df_all[df_all['target'] == 1]

df_majority_downsampled = resample(
    df_majority,
    replace=False,
    n_samples=len(df_minority),
    random_state=42
)

df_balanced = pd.concat([df_majority_downsampled, df_minority])

X_bal = df_balanced.drop(columns=['target'])
y_bal = df_balanced['target']

# 학습용 80% / 테스트용 20%만 분할
X_train, X_test, y_train, y_test = train_test_split(
    X_bal, y_bal, test_size=0.2, stratify=y_bal, random_state=42
)

# 모델 학습
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

#테스트 데이터 예측 및 성능 평가
test_pred = rf_model.predict(X_test)
print("테스트 데이터 성능 평가:")
print(classification_report(y_test, test_pred))

테스트 데이터 성능 평가:
              precision    recall  f1-score   support

           0       0.66      0.68      0.67       322
           1       0.67      0.65      0.66       321

    accuracy                           0.67       643
   macro avg       0.67      0.67      0.67       643
weighted avg       0.67      0.67      0.67       643



In [13]:
## 언더샘플링 + 하이퍼파라미터 + 임계값

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.utils import resample
import pandas as pd

drop_cols = ['DS1_ID', 'DS1_SEX', 'DS1_GLU0', 'DS1_HBA1C']

X = male_df.drop(columns=drop_cols + ['target'])
y = male_df['target']


df_all = X.copy()
df_all['target'] = y
df_majority = df_all[df_all['target'] == 0]
df_minority = df_all[df_all['target'] == 1]
df_majority_down = resample(df_majority, replace=False, n_samples=len(df_minority), random_state=42)
df_balanced = pd.concat([df_majority_down, df_minority])


X_bal = df_balanced.drop(columns=['target'])
y_bal = df_balanced['target']


X_train, X_test, y_train, y_test = train_test_split(X_bal, y_bal, test_size=0.2, stratify=y_bal, random_state=42)


rf4 = RandomForestClassifier(
    n_estimators=200,
    max_depth=300,
    min_samples_leaf=60,
    random_state=42
)
rf4.fit(X_train, y_train)
pred4 = rf4.predict(X_test)
print(" [4] 튜닝 적용 (max_depth=10, min_samples_leaf=3):")
print(classification_report(y_test, pred4))


 [4] 튜닝 적용 (max_depth=10, min_samples_leaf=3):
              precision    recall  f1-score   support

           0       0.72      0.68      0.70       322
           1       0.69      0.74      0.71       321

    accuracy                           0.71       643
   macro avg       0.71      0.71      0.71       643
weighted avg       0.71      0.71      0.71       643



In [15]:
## 언더샘플링 + 하이퍼파라미터 + 임계값 + threshold

In [16]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from statsmodels.stats.outliers_influence import variance_inflation_factor
import matplotlib.pyplot as plt

# -------------------------------------------
# 1. 변수 제거 + 스케일링
drop_cols = ['DS1_ID', 'DS1_SEX', 'DS1_GLU0', 'DS1_HBA1C']
X = male_df.drop(columns=drop_cols + ['target'])
y = male_df['target']

scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# -------------------------------------------
# 2. VIF 기반 변수 제거
def calculate_vif(X, thresh=10.0):
    variables = X.columns.tolist()
    while True:
        vif = [variance_inflation_factor(X[variables].values, i) for i in range(len(variables))]
        max_vif = max(vif)
        if max_vif > thresh:
            max_index = vif.index(max_vif)
            print(f"제거된 변수: {variables[max_index]} (VIF: {max_vif:.2f})")
            variables.pop(max_index)
        else:
            break
    return X[variables]

X_vif_filtered = calculate_vif(X_scaled)

# -------------------------------------------
# 3. 언더샘플링
df_all = X_vif_filtered.copy()
df_all['target'] = y

df_majority = df_all[df_all['target'] == 0]
df_minority = df_all[df_all['target'] == 1]

df_majority_downsampled = resample(
    df_majority,
    replace=False,
    n_samples=len(df_minority),
    random_state=42
)

df_balanced = pd.concat([df_majority_downsampled, df_minority])
X_bal = df_balanced.drop(columns=['target'])
y_bal = df_balanced['target']

# -------------------------------------------
# 4. 학습/테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X_bal, y_bal, test_size=0.2, stratify=y_bal, random_state=42
)

# -------------------------------------------
# 5. 랜덤포레스트 학습
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
rf_model.fit(X_train, y_train)
y_proba_rf = rf_model.predict_proba(X_test)[:, 1]

# -------------------------------------------
# 6. Threshold 조정 실험
thresholds = np.arange(0.30, 0.51, 0.01)
results = []

for threshold in thresholds:
    y_pred = (y_proba_rf >= threshold).astype(int)
    results.append({
        'Threshold': round(threshold, 2),
        'Accuracy': round(accuracy_score(y_test, y_pred), 3),
        'Precision': round(precision_score(y_test, y_pred), 3),
        'Recall': round(recall_score(y_test, y_pred), 3),
        'F1 Score': round(f1_score(y_test, y_pred), 3)
    })

results_df = pd.DataFrame(results)
print("\n Threshold별 성능 비교:")
print(results_df)

제거된 변수: DS1_WALK (VIF: 653212061.71)
제거된 변수: DS1_WALKT (VIF: 2404511.08)
제거된 변수: DS1_WAIST (VIF: 141.95)
제거된 변수: DS1_BODYFAT (VIF: 34.30)

 Threshold별 성능 비교:
    Threshold  Accuracy  Precision  Recall  F1 Score
0        0.30     0.580      0.544   0.978     0.699
1        0.31     0.586      0.549   0.969     0.700
2        0.32     0.591      0.552   0.960     0.701
3        0.33     0.605      0.561   0.960     0.708
4        0.34     0.611      0.566   0.947     0.709
5        0.35     0.624      0.576   0.931     0.712
6        0.36     0.639      0.588   0.922     0.718
7        0.37     0.645      0.594   0.916     0.721
8        0.38     0.645      0.596   0.900     0.717
9        0.39     0.644      0.597   0.879     0.711
10       0.40     0.649      0.604   0.860     0.710
11       0.41     0.658      0.613   0.850     0.713
12       0.42     0.659      0.617   0.838     0.711
13       0.43     0.652      0.614   0.816     0.701
14       0.44     0.655      0.618   0.807     

In [18]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, classification_report
)
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from itertools import product


drop_cols = ['DS1_ID', 'DS1_SEX', 'DS1_GLU0', 'DS1_HBA1C',
             'DS1_WALK', 'DS1_WALKT', 'DS1_WAIST', 'DS1_BODYFAT', 'DS1_MARRY_RE']
X = male_df.drop(columns=drop_cols + ['target'])
y = male_df['target']


df_all = X.copy()
df_all['target'] = y
df_majority = df_all[df_all['target'] == 0]
df_minority = df_all[df_all['target'] == 1]
df_majority_down = resample(df_majority, replace=False, n_samples=len(df_minority), random_state=42)
df_balanced = pd.concat([df_majority_down, df_minority])
X_bal = df_balanced.drop(columns=['target'])
y_bal = df_balanced['target']

scaler = StandardScaler()
X_bal_scaled = pd.DataFrame(scaler.fit_transform(X_bal), columns=X_bal.columns) #-> 트리구조는 필요 없음 

X_trainval, X_test, y_trainval, y_test = train_test_split(
    X_bal_scaled, y_bal, test_size=0.2, stratify=y_bal, random_state=42
)
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=0.25, stratify=y_trainval, random_state=42
)
X_train_full = pd.concat([X_train, X_val])
y_train_full = pd.concat([y_train, y_val])


param_grid = {
    'n_estimators': [200, 300],
    'max_depth': [10, 30, 50],
    'min_samples_leaf': [10, 50],
    'max_features': ['sqrt', 0.5],
    'class_weight': ['balanced']
}
param_combinations = list(product(
    param_grid['n_estimators'],
    param_grid['max_depth'],
    param_grid['min_samples_leaf'],
    param_grid['max_features'],
    param_grid['class_weight']
))


results = []
for i, (n_est, depth, leaf, feat, weight) in enumerate(param_combinations):
    model = RandomForestClassifier(
        n_estimators=n_est,
        max_depth=depth,
        min_samples_leaf=leaf,
        max_features=feat,
        class_weight=weight,
        random_state=42,
        n_jobs=-1
    )
    model.fit(X_train_full, y_train_full)
    y_proba = model.predict_proba(X_test)[:, 1]
    y_pred = (y_proba >= 0.5).astype(int)

    results.append({
        'Model_ID': f"RF_{i+1}",
        'n_estimators': n_est,
        'max_depth': depth,
        'min_samples_leaf': leaf,
        'max_features': feat,
        'class_weight': weight,
        'Accuracy': round(accuracy_score(y_test, y_pred), 3),
        'Precision': round(precision_score(y_test, y_pred), 3),
        'Recall': round(recall_score(y_test, y_pred), 3),
        'F1': round(f1_score(y_test, y_pred), 3),
        'AUC': round(roc_auc_score(y_test, y_proba), 3)
    })


results_df = pd.DataFrame(results)
results_df['AUC >= 0.8'] = results_df['AUC'] >= 0.8
results_df = results_df.sort_values(by='AUC', ascending=False)

print("\n랜덤포레스트 자동 튜닝 결과 (언더샘플링 + StandardScaler):")
print(results_df)


best_row = results_df.iloc[0]
print(f"\n최고 AUC 모델: {best_row['Model_ID']} (AUC={best_row['AUC']})")

best_model = RandomForestClassifier(
    n_estimators=int(best_row['n_estimators']),
    max_depth=int(best_row['max_depth']),
    min_samples_leaf=int(best_row['min_samples_leaf']),
    max_features=best_row['max_features'],
    class_weight=best_row['class_weight'],
    random_state=42,
    n_jobs=-1
)
best_model.fit(X_train_full, y_train_full)
y_proba = best_model.predict_proba(X_test)[:, 1]

thresholds = np.arange(0.30, 0.51, 0.01)
thresh_results = []

for t in thresholds:
    y_pred = (y_proba >= t).astype(int)
    thresh_results.append({
        'Threshold': round(t, 2),
        'Accuracy': round(accuracy_score(y_test, y_pred), 3),
        'Precision': round(precision_score(y_test, y_pred), 3),
        'Recall': round(recall_score(y_test, y_pred), 3),
        'F1 Score': round(f1_score(y_test, y_pred), 3),
        'AUC': round(roc_auc_score(y_test, y_proba), 3)
    })

thresh_df = pd.DataFrame(thresh_results)

print(thresh_df)


final_pred = (y_proba >= 0.5).astype(int)
print("\nClassification Report (threshold=0.5 기준):")
print(classification_report(y_test, final_pred))


랜덤포레스트 자동 튜닝 결과 (언더샘플링 + StandardScaler):
   Model_ID  n_estimators  max_depth  min_samples_leaf max_features  \
14    RF_15           300         10                50         sqrt   
2      RF_3           200         10                50         sqrt   
22    RF_23           300         50                50         sqrt   
6      RF_7           200         30                50         sqrt   
18    RF_19           300         30                50         sqrt   
10    RF_11           200         50                50         sqrt   
12    RF_13           300         10                10         sqrt   
3      RF_4           200         10                50          0.5   
20    RF_21           300         50                10         sqrt   
7      RF_8           200         30                50          0.5   
11    RF_12           200         50                50          0.5   
16    RF_17           300         30                10         sqrt   
19    RF_20           300         

## xgboost

In [20]:
from xgboost import XGBClassifier

# 하이퍼파라미터 조합 정의
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}
param_combinations = list(product(
    param_grid['n_estimators'],
    param_grid['max_depth'],
    param_grid['learning_rate'],
    param_grid['subsample'],
    param_grid['colsample_bytree']
))[:20]

results = []
best_auc = 0
best_model = None
best_pred_proba = None

for i, (n_est, depth, lr, subsample, colsample) in enumerate(param_combinations):
    model = XGBClassifier(
        n_estimators=n_est,
        max_depth=depth,
        learning_rate=lr,
        subsample=subsample,
        colsample_bytree=colsample,
        use_label_encoder=False,
        eval_metric='logloss',
        random_state=42,
        n_jobs=-1
    )
    model.fit(X_train_full, y_train_full)
    y_proba_test = model.predict_proba(X_test)[:, 1]
    y_pred_test = (y_proba_test >= 0.5).astype(int)
    auc = roc_auc_score(y_test, y_proba_test)

    if auc > best_auc:
        best_auc = auc
        best_model = model
        best_pred_proba = y_proba_test

    results.append({
        'Model_ID': f"XGB_{i+1}",
        'n_estimators': n_est,
        'max_depth': depth,
        'learning_rate': lr,
        'subsample': subsample,
        'colsample_bytree': colsample,
        'Test_Accuracy': round(accuracy_score(y_test, y_pred_test), 3),
        'Test_Precision': round(precision_score(y_test, y_pred_test), 3),
        'Test_Recall': round(recall_score(y_test, y_pred_test), 3),
        'Test_F1': round(f1_score(y_test, y_pred_test), 3),
        'Test_AUC': round(auc, 3)
    })


results_df = pd.DataFrame(results)
results_df['AUC >= 0.8'] = results_df['Test_AUC'] >= 0.8
results_df = results_df.sort_values(by='Test_AUC', ascending=False)

print("\n상위 20개 XGBoost 튜닝 결과:")
print(results_df)


y_val_proba = best_model.predict_proba(X_val)[:, 1]
y_val_pred = (y_val_proba >= 0.5).astype(int)
print(classification_report(y_val, y_val_pred))
print(f"AUC (Validation): {roc_auc_score(y_val, y_val_proba):.3f}")


thresholds = np.arange(0.40, 0.51, 0.01)
best_results = []

for t in thresholds:
    pred = (best_pred_proba >= t).astype(int)
    best_results.append({
        'Threshold': round(t, 2),
        'Accuracy': round(accuracy_score(y_test, pred), 3),
        'Precision': round(precision_score(y_test, pred), 3),
        'Recall': round(recall_score(y_test, pred), 3),
        'F1': round(f1_score(y_test, pred), 3),
        'AUC': round(best_auc, 3)
    })

best_thresh_df = pd.DataFrame(best_results)
print(best_thresh_df)

print("\n최종 Classification Report (Test, threshold = 0.5):")
final_pred = (best_pred_proba >= 0.5).astype(int)
print(classification_report(y_test, final_pred))

C:\Users\JEONGHEE\AppData\Roaming\Python\Python312\site-packages\xgboost\training.py:183: UserWarning: [19:17:05] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\JEONGHEE\AppData\Roaming\Python\Python312\site-packages\xgboost\training.py:183: UserWarning: [19:17:05] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\JEONGHEE\AppData\Roaming\Python\Python312\site-packages\xgboost\training.py:183: UserWarning: [19:17:05] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\JEONGHEE\AppData\Roaming\Python\Python312\site-packages\xgboost\training.py:183: UserWarning: [19:17:06] WARNING: C:\actions-runner\_work\xgboost\xgboost\


상위 20개 XGBoost 튜닝 결과:
   Model_ID  n_estimators  max_depth  learning_rate  subsample  \
4     XGB_5           100          3           0.10        0.8   
5     XGB_6           100          3           0.10        0.8   
8     XGB_9           100          5           0.01        0.8   
2     XGB_3           100          3           0.01        1.0   
6     XGB_7           100          3           0.10        1.0   
11   XGB_12           100          5           0.01        1.0   
7     XGB_8           100          3           0.10        1.0   
9    XGB_10           100          5           0.01        0.8   
1     XGB_2           100          3           0.01        0.8   
0     XGB_1           100          3           0.01        0.8   
10   XGB_11           100          5           0.01        1.0   
3     XGB_4           100          3           0.01        1.0   
16   XGB_17           100          7           0.01        0.8   
15   XGB_16           100          5           0.10  

## 로지스틱회귀

In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 1. 독립변수(X), 종속변수(y) 설정
X = male_df.drop(columns=['target', 'DS1_SEX', 'DS1_GLU0', 'DS1_HBA1C', 'DS1_ID'])
y = male_df['target']

# 2. 스케일링
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# 3. VIF 기준 변수 제거 함수 정의
def calculate_vif(X, thresh=10.0):
    variables = X.columns.tolist()
    dropped = []

    while True:
        vif = [variance_inflation_factor(X[variables].values, i) for i in range(len(variables))]
        max_vif = max(vif)
        if max_vif > thresh:
            max_index = vif.index(max_vif)
            print(f"제거된 변수: {variables[max_index]} (VIF: {max_vif:.2f})")
            dropped.append(variables[max_index])
            variables.pop(max_index)
        else:
            break

    print("\n최종 남은 변수:")
    print(variables)
    return X[variables], dropped




# 4. VIF 기반 변수 제거 적용
X_vif_filtered, dropped_vars = calculate_vif(X_scaled)

# 5. 언더샘플링
rus = RandomUnderSampler(random_state=42)
X_resampled_vif, y_resampled_vif = rus.fit_resample(X_vif_filtered, y)
print("언더샘플링 후 클래스 비율:\n", y_resampled_vif.value_counts())

# # 6. 훈련/테스트 분리
# X_train_vif, X_test_vif, y_train_vif, y_test_vif = train_test_split(
#     X_resampled_vif, y_resampled_vif, test_size=0.2, random_state=42, stratify=y_resampled_vif
# )
# print("훈련 데이터 크기:", X_train_vif.shape)
# print("테스트 데이터 크기:", X_test_vif.shape)


# 6. 훈련/검증/테스트 분리 (6:2:2 비율)
# 전체에서 60% train, 40% temp (val + test)
X_train_vif, X_temp_vif, y_train_vif, y_temp_vif = train_test_split(
    X_resampled_vif, y_resampled_vif, test_size=0.4, random_state=42, stratify=y_resampled_vif
)

# temp에서 다시 50% val, 50% test → 각각 전체의 20%
X_val_vif, X_test_vif, y_val_vif, y_test_vif = train_test_split(
    X_temp_vif, y_temp_vif, test_size=0.5, random_state=42, stratify=y_temp_vif
)

print("훈련 데이터 크기:", X_train_vif.shape)
print("검증 데이터 크기:", X_val_vif.shape)
print("테스트 데이터 크기:", X_test_vif.shape)

# 7. 로지스틱 회귀 모델 학습 및 예측
model_vif = LogisticRegression(max_iter=1000, random_state=42)
model_vif.fit(X_train_vif, y_train_vif)
y_pred_vif = model_vif.predict(X_test_vif)

# 8. 성능 평가
accuracy = accuracy_score(y_test_vif, y_pred_vif)
cm = confusion_matrix(y_test_vif, y_pred_vif)
report = classification_report(y_test_vif, y_pred_vif)

print(f"\n정확도(Accuracy): {accuracy:.3f}")
print("\n혼동 행렬:\n", cm)
print("\n분류 리포트:\n", report)

제거된 변수: DS1_WALK (VIF: 653212061.71)
제거된 변수: DS1_WALKT (VIF: 2404511.08)
제거된 변수: DS1_WAIST (VIF: 141.95)
제거된 변수: DS1_BODYFAT (VIF: 34.30)

최종 남은 변수:
['DS1_AGE', 'DS1_HTN', 'DS1_FDM', 'DS1_EXER', 'DS1_WALKFQ', 'DS1_HIP', 'DS1_PULSE', 'DS1_SBP', 'DS1_DBP', 'DS1_BMI', 'DS1_PBF', 'DS1_MUSCLE', 'DS1_VISFAT', 'DS1_WHR', 'DS1_INCOME_RE', 'DS1_EDU_RE', 'DS1_MARRY_RE', 'total_exer', 'DS1_SMOKE_RE', 'DS1_DRINK_RE']
언더샘플링 후 클래스 비율:
 target
0    1607
1    1607
Name: count, dtype: int64
훈련 데이터 크기: (1928, 20)
검증 데이터 크기: (643, 20)
테스트 데이터 크기: (643, 20)

정확도(Accuracy): 0.647

혼동 행렬:
 [[212 110]
 [117 204]]

분류 리포트:
               precision    recall  f1-score   support

           0       0.64      0.66      0.65       322
           1       0.65      0.64      0.64       321

    accuracy                           0.65       643
   macro avg       0.65      0.65      0.65       643
weighted avg       0.65      0.65      0.65       643



In [23]:
from sklearn.model_selection import cross_val_score, StratifiedKFold

model_vif = LogisticRegression(max_iter=1000, random_state=42)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scores = cross_val_score(model_vif, X_resampled_vif, y_resampled_vif, cv=cv, scoring='accuracy')
print("각 Fold 정확도:", scores)
print("평균 정확도:", scores.mean())

각 Fold 정확도: [0.62830482 0.66096423 0.65474339 0.64230171 0.62928349]
평균 정확도: 0.6431195283014297


In [25]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 1. 모델 학습
model = LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)
model.fit(X_train_vif, y_train_vif)

# 2. 확률 예측
y_proba = model.predict_proba(X_test_vif)[:, 1]

# 3. threshold 리스트 (0.3 ~ 0.5, 0.01 간격)
thresholds = np.arange(0.30, 0.51, 0.01)

# 4. 결과 저장
results = []

for threshold in thresholds:
    y_pred = (y_proba >= threshold).astype(int)
    acc = accuracy_score(y_test_vif, y_pred)
    prec = precision_score(y_test_vif, y_pred)
    rec = recall_score(y_test_vif, y_pred)
    f1 = f1_score(y_test_vif, y_pred)
    results.append({
        'Threshold': round(threshold, 2),
        'Accuracy': round(acc, 3),
        'Precision': round(prec, 3),
        'Recall': round(rec, 3),
        'F1 Score': round(f1, 3)
    })

# 5. 결과 보기
results_df = pd.DataFrame(results)
print("\nThreshold별 성능 비교:")
print(results_df)


Threshold별 성능 비교:
    Threshold  Accuracy  Precision  Recall  F1 Score
0        0.30     0.619      0.570   0.963     0.716
1        0.31     0.628      0.577   0.953     0.719
2        0.32     0.627      0.578   0.938     0.715
3        0.33     0.636      0.584   0.938     0.720
4        0.34     0.645      0.592   0.935     0.725
5        0.35     0.650      0.597   0.919     0.724
6        0.36     0.656      0.604   0.907     0.725
7        0.37     0.659      0.609   0.888     0.722
8        0.38     0.663      0.614   0.875     0.721
9        0.39     0.667      0.620   0.860     0.721
10       0.40     0.670      0.625   0.847     0.720
11       0.41     0.678      0.635   0.835     0.721
12       0.42     0.672      0.633   0.816     0.713
13       0.43     0.663      0.629   0.791     0.701
14       0.44     0.669      0.640   0.769     0.699
15       0.45     0.670      0.644   0.760     0.697
16       0.46     0.677      0.657   0.738     0.695
17       0.47     0.669    

# 여자

## 랜덤포레스트 : 6:2:2 + 언더샘플링 + vif + 하이퍼파라미터 + threshold

In [31]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, classification_report
)
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from itertools import product

# ✅ 1. 데이터 불러오기
female_df = pd.read_csv("C:/Users/JEONGHEE/Desktop/당뇨병플젝/female.csv")

# ✅ 2. 제거할 변수 정의 후 제거
remove_cols = ['DS1_ID', 'DS1_SEX', 'DS1_GLU0', 'DS1_HBA1C',
               'DS1_WALK', 'DS1_WALKT', 'DS1_WAIST', 'DS1_BODYFAT', 'DS1_MARRY_RE']
X = female_df.drop(columns=remove_cols + ['target'])
y = female_df['target']

# ✅ 3. 언더샘플링
df_all = X.copy()
df_all['target'] = y
df_majority = df_all[df_all['target'] == 0]
df_minority = df_all[df_all['target'] == 1]
df_majority_down = resample(df_majority, replace=False, n_samples=len(df_minority), random_state=42)
df_balanced = pd.concat([df_majority_down, df_minority])
X_bal = df_balanced.drop(columns=['target'])
y_bal = df_balanced['target']




scaler = StandardScaler()
X_bal_scaled = pd.DataFrame(scaler.fit_transform(X_bal), columns=X_bal.columns)


X_trainval, X_test, y_trainval, y_test = train_test_split(
    X_bal_scaled, y_bal, test_size=0.2, stratify=y_bal, random_state=42
)
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=0.25, stratify=y_trainval, random_state=42
)
X_train_full = pd.concat([X_train, X_val])
y_train_full = pd.concat([y_train, y_val])



param_grid = {
    'n_estimators': [200, 300],
    'max_depth': [5, 10, 15, 20, 30],
    'min_samples_leaf': [5, 10, 20],
    'max_features': ['sqrt', 0.5],
    'class_weight': ['balanced']
}
param_combinations = list(product(
    param_grid['n_estimators'],
    param_grid['max_depth'],
    param_grid['min_samples_leaf'],
    param_grid['max_features'],
    param_grid['class_weight']
))[:20]



results = []
best_auc = 0
best_model = None
best_pred_proba = None

for i, (n_est, depth, leaf, feat, weight) in enumerate(param_combinations):
    model = RandomForestClassifier(
        n_estimators=n_est,
        max_depth=depth,
        min_samples_leaf=leaf,
        max_features=feat,
        class_weight=weight,
        random_state=42,
        n_jobs=-1
    )
    model.fit(X_train_full, y_train_full)

    y_proba_test = model.predict_proba(X_test)[:, 1]
    y_pred_test = (y_proba_test >= 0.5).astype(int)
    auc = roc_auc_score(y_test, y_proba_test)

    if auc > best_auc:
        best_auc = auc
        best_model = model
        best_pred_proba = y_proba_test

    results.append({
        'Model_ID': f"RF_F_{i+1}",
        'n_estimators': n_est,
        'max_depth': depth,
        'min_samples_leaf': leaf,
        'max_features': feat,
        'class_weight': weight,
        'Test_Accuracy': round(accuracy_score(y_test, y_pred_test), 3),
        'Test_Precision': round(precision_score(y_test, y_pred_test), 3),
        'Test_Recall': round(recall_score(y_test, y_pred_test), 3),
        'Test_F1': round(f1_score(y_test, y_pred_test), 3),
        'Test_AUC': round(auc, 3)
    })

results_df = pd.DataFrame(results)
results_df['AUC >= 0.8'] = results_df['Test_AUC'] >= 0.8
results_df = results_df.sort_values(by='Test_AUC', ascending=False)

print("\n상위 20개 랜덤포레스트 튜닝 결과 (여성):")
print(results_df)


print("\n최고 AUC 모델의 Validation Set 성능:")
y_val_proba = best_model.predict_proba(X_val)[:, 1]
y_val_pred = (y_val_proba >= 0.5).astype(int)
print(classification_report(y_val, y_val_pred))
print(f"AUC (Validation): {roc_auc_score(y_val, y_val_proba):.3f}")


print("\nTest 기준 Threshold 실험:")
thresholds = np.arange(0.40, 0.51, 0.01)
best_results = []

for t in thresholds:
    pred = (best_pred_proba >= t).astype(int)
    best_results.append({
        'Threshold': round(t, 2),
        'Accuracy': round(accuracy_score(y_test, pred), 3),
        'Precision': round(precision_score(y_test, pred), 3),
        'Recall': round(recall_score(y_test, pred), 3),
        'F1': round(f1_score(y_test, pred), 3),
        'AUC': round(best_auc, 3)
    })

thresh_df = pd.DataFrame(best_results)
print(thresh_df)



print("\n최종 Classification Report (Test):")
final_pred = (best_pred_proba >= 0.5).astype(int)
print(classification_report(y_test, final_pred))



상위 20개 랜덤포레스트 튜닝 결과 (여성):
   Model_ID  n_estimators  max_depth  min_samples_leaf max_features  \
9   RF_F_10           200         10                10          0.5   
11  RF_F_12           200         10                20          0.5   
10  RF_F_11           200         10                20         sqrt   
7    RF_F_8           200         10                 5          0.5   
17  RF_F_18           200         15                20          0.5   
15  RF_F_16           200         15                10          0.5   
13  RF_F_14           200         15                 5          0.5   
19  RF_F_20           200         20                 5          0.5   
16  RF_F_17           200         15                20         sqrt   
8    RF_F_9           200         10                10         sqrt   
6    RF_F_7           200         10                 5         sqrt   
12  RF_F_13           200         15                 5         sqrt   
14  RF_F_15           200         15              

## xgboost : 6:2:2 + 언더샘플링 + vif + 하이퍼파라미터 + threshold

In [32]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, classification_report
)
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from itertools import product
female_df = pd.read_csv("C:/Users/JEONGHEE/Desktop/당뇨병플젝/female.csv")



remove_cols = ['DS1_ID', 'DS1_SEX', 'DS1_GLU0', 'DS1_HBA1C',
               'DS1_WALK', 'DS1_WALKT', 'DS1_WAIST', 'DS1_BODYFAT', 'DS1_MARRY_RE']
X = female_df.drop(columns=remove_cols + ['target'])
y = female_df['target']



X['target'] = y
majority = X[X['target'] == 0]
minority = X[X['target'] == 1]
majority_down = resample(majority, replace=False, n_samples=len(minority), random_state=42)
X_balanced = pd.concat([majority_down, minority])
X_bal = X_balanced.drop(columns=['target'])
y_bal = X_balanced['target']


scaler = StandardScaler()
X_bal_scaled = pd.DataFrame(scaler.fit_transform(X_bal), columns=X_bal.columns) #-> 로지스틱 회귀는 스케일링 진행



X_trainval, X_test, y_trainval, y_test = train_test_split(
    X_bal_scaled, y_bal, test_size=0.2, stratify=y_bal, random_state=42
)
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=0.25, stratify=y_trainval, random_state=42
)
X_train_full = pd.concat([X_train, X_val])
y_train_full = pd.concat([y_train, y_val])



param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}
param_combinations = list(product(
    param_grid['n_estimators'],
    param_grid['max_depth'],
    param_grid['learning_rate'],
    param_grid['subsample'],
    param_grid['colsample_bytree']
))[:20]

results = []
best_auc = 0
best_model = None
best_pred_proba = None

for i, (n_est, depth, lr, subsample, colsample) in enumerate(param_combinations):
    model = XGBClassifier(
        n_estimators=n_est,
        max_depth=depth,
        learning_rate=lr,
        subsample=subsample,
        colsample_bytree=colsample,
        use_label_encoder=False,
        eval_metric='logloss',
        random_state=42,
        n_jobs=-1
    )
    model.fit(X_train_full, y_train_full)
    y_proba_test = model.predict_proba(X_test)[:, 1]
    y_pred_test = (y_proba_test >= 0.5).astype(int)
    auc = roc_auc_score(y_test, y_proba_test)

    if auc > best_auc:
        best_auc = auc
        best_model = model
        best_pred_proba = y_proba_test

    results.append({
        'Model_ID': f"XGB_F_{i+1}",
        'n_estimators': n_est,
        'max_depth': depth,
        'learning_rate': lr,
        'subsample': subsample,
        'colsample_bytree': colsample,
        'Test_Accuracy': round(accuracy_score(y_test, y_pred_test), 3),
        'Test_Precision': round(precision_score(y_test, y_pred_test), 3),
        'Test_Recall': round(recall_score(y_test, y_pred_test), 3),
        'Test_F1': round(f1_score(y_test, y_pred_test), 3),
        'Test_AUC': round(auc, 3)
    })

results_df = pd.DataFrame(results)
results_df['AUC >= 0.8'] = results_df['Test_AUC'] >= 0.8
results_df = results_df.sort_values(by='Test_AUC', ascending=False)

print("\n상위 20개 XGBoost 튜닝 결과 (여성):")
print(results_df)


print("\n최고 AUC 모델의 Validation Set 성능:")
y_val_proba = best_model.predict_proba(X_val)[:, 1]
y_val_pred = (y_val_proba >= 0.5).astype(int)
print(classification_report(y_val, y_val_pred))
print(f"AUC (Validation): {roc_auc_score(y_val, y_val_proba):.3f}")


print("\n Test 기준 Threshold 실험:")
thresholds = np.arange(0.40, 0.51, 0.01)
best_results = []
for t in thresholds:
    pred = (best_pred_proba >= t).astype(int)
    best_results.append({
        'Threshold': round(t, 2),
        'Accuracy': round(accuracy_score(y_test, pred), 3),
        'Precision': round(precision_score(y_test, pred), 3),
        'Recall': round(recall_score(y_test, pred), 3),
        'F1': round(f1_score(y_test, pred), 3),
        'AUC': round(best_auc, 3)
    })
thresh_df = pd.DataFrame(best_results)
print(thresh_df)



print("\n최종 Classification Report (Test):")
final_pred = (best_pred_proba >= 0.5).astype(int)
print(classification_report(y_test, final_pred))

C:\Users\JEONGHEE\AppData\Roaming\Python\Python312\site-packages\xgboost\training.py:183: UserWarning: [19:24:12] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\JEONGHEE\AppData\Roaming\Python\Python312\site-packages\xgboost\training.py:183: UserWarning: [19:24:13] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\JEONGHEE\AppData\Roaming\Python\Python312\site-packages\xgboost\training.py:183: UserWarning: [19:24:13] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\JEONGHEE\AppData\Roaming\Python\Python312\site-packages\xgboost\training.py:183: UserWarning: [19:24:13] WARNING: C:\actions-runner\_work\xgboost\xgboost\


상위 20개 XGBoost 튜닝 결과 (여성):
    Model_ID  n_estimators  max_depth  learning_rate  subsample  \
5    XGB_F_6           100          3           0.10        0.8   
12  XGB_F_13           100          5           0.10        0.8   
7    XGB_F_8           100          3           0.10        1.0   
4    XGB_F_5           100          3           0.10        0.8   
6    XGB_F_7           100          3           0.10        1.0   
14  XGB_F_15           100          5           0.10        1.0   
17  XGB_F_18           100          7           0.01        0.8   
16  XGB_F_17           100          7           0.01        0.8   
15  XGB_F_16           100          5           0.10        1.0   
8    XGB_F_9           100          5           0.01        0.8   
9   XGB_F_10           100          5           0.01        0.8   
13  XGB_F_14           100          5           0.10        0.8   
10  XGB_F_11           100          5           0.01        1.0   
18  XGB_F_19           100        